In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
def process_image(image_path, IMG_SIZE=224):

    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image, channels=3, expand_animations= False)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
    return image

In [3]:
images_loc = []
labels = []
for i in os.listdir("/kaggle/input/forest-fire-dataset/Forest Fire Dataset/Training/nofire"):
    images_loc.append(process_image("/kaggle/input/forest-fire-dataset/Forest Fire Dataset/Training/nofire"+"/"+i))
    labels.append(0)
for i in os.listdir("/kaggle/input/forest-fire-dataset/Forest Fire Dataset/Training/fire"):
    images_loc.append(process_image("/kaggle/input/forest-fire-dataset/Forest Fire Dataset/Training/fire"+"/"+i))
    labels.append(1)

NameError: name 'tf' is not defined

In [ ]:
classnames = np.array(["nofire", "fire"])

In [ ]:
images_loc = np.array(images_loc)
labels = np.array(labels)

In [ ]:
images_loc.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_testy, y_train, y_testy = train_test_split(images_loc, labels, test_size = 0.3)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_testy, y_testy, test_size = 0.5)

In [ ]:
len(X_test), len(y_test)

In [ ]:
import matplotlib.pyplot as plt
def show_25_images(images):
    plt.figure(figsize=(10,10))
    for i in range(25):
        ax=plt.subplot(5,5,i+1)
        plt.imshow(images[i])
#         plt.title(classnames[labels[i]])
        plt.axis("off")

In [ ]:
show_25_images(X_train)

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, ReLU, DepthwiseConv2D, Concatenate, Reshape, Lambda, UpSampling2D, Activation, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, BatchNormalization, LSTM, Conv1D, MaxPooling1D, Conv2D, Input, Concatenate, Add, PReLU, AveragePooling2D
import tensorflow as tf


## Using Custom cnn

In [ ]:
def custom_cnn(X1, X2, X3):
    inputs = Input(shape=(X1, X2, X3))
    C1 = Conv2D(128, (5, 5), activation='relu', padding='valid')(inputs)
    P1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(C1)
    C2 = Conv2D(64, (3, 3), activation='relu', padding='same')(P1)
    C3 = Conv2D(64, (3, 3), activation='relu', padding='same')(C2)
    P2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(C3)
    C4 = Conv2D(32, (3, 3), activation='relu', padding='same')(P2)
    C5 = Conv2D(32, (3, 3), activation='relu', padding='same')(C4)
    P3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(C5)
    F1 = Flatten()(P3)
    D1 = Dense(1024, activation='relu')(F1)
    DR1 = Dropout(0.5)(D1)
    D2 = Dense(512, activation='relu')(DR1)
    DR2 = Dropout(0.5)(D2)
    outputs = Dense(1, activation='sigmoid')(DR2)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
m = custom_cnn(224,224,3)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
pred = m.fit(X_train, y_train,
             batch_size=32,epochs=25,
             validation_data=(X_val, y_val),
             callbacks=[early_stopping])

In [ ]:
y_pred = m.predict(X_test)
binary_predictions = (y_pred > 0.5).astype(int)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def results(y_test, y_pred):    
    cm = confusion_matrix(y_test, y_pred)
    TP,TN,FP,FN = cm[0][0],cm[1][1],cm[0][1],cm[1][0]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    SE = TP / (TP + FN)
    SP = TN / (TN + FP)
    ACC = (TP + TN) / (TP + FP + FN + TN)
    DSC = (2 * TP) / (2 * TP + FP + FN)
    Precision = TP / (TP + FP)
    Recall = TP / (TP + FN)
    print(f"SE: {SE:.2f}")
    print(f"SP: {SP:.2f}")
    print(f"ACC: {ACC:.2f}")
    print(f"DSC: {DSC:.2f}")
    print(f"Precision: {Precision:.2f}")
    print(f"Recall: {Recall:.2f}")

In [ ]:
results(y_test,binary_predictions)

## ATT Squeeze U-Net

In [ ]:
def channel_shuffle(x, groups):
    batch_size, height, width, channels = x.shape
    channel_per_group = channels // groups
    x = Reshape((height, width, groups, channel_per_group))(x)
    x = tf.transpose(x, perm=[0, 1, 2, 4, 3])
    x = Reshape((height, width, channels))(x)
    return x

def fire_module(x, s1, e1, e3):
    s1x = Conv2D(s1, kernel_size=1, padding='same')(x)
    s1x = ReLU()(s1x)
    e1x = Conv2D(e1, kernel_size=1, padding='same')(s1x)
    e3x = DepthwiseConv2D(kernel_size=3, padding='same')(s1x)
    e3x = Conv2D(e3, kernel_size=1, padding='same')(e3x)
    total_channels = e1 + e3
    x = Concatenate()([e1x, e3x])
    x = channel_shuffle(x, groups=2)
    x = ReLU()(x)
    
    return x

def defire_module(x, s1, e3):
    # First 1x1 convolution
    e1x = Conv2D(s1, kernel_size=(1, 1), padding='same')(x)
    e1x = Activation('relu')(e1x)
    
    # First 3x3 convolution (without changing the number of filters)
    e1x = Conv2D(s1, kernel_size=(3, 3), padding='same')(e1x)
    e1x = Activation('relu')(e1x)
    e3x = Conv2D(s1, kernel_size=(3, 3), padding='same')(e1x)
    e3x = Activation('relu')(e3x)
    squeeze = Conv2D(s1, kernel_size=(1, 1), padding='same')(e3x)
    squeeze = Activation('relu')(squeeze)

    # Upsampling and 3x3 Convolution to increase the number of filters
    upsampled = UpSampling2D(size=(2, 2))(squeeze)
    e3x_final = Conv2D(e3, kernel_size=(3, 3), padding='same')(upsampled)
    e3x_final = Activation('relu')(e3x_final)

    return e3x

import tensorflow.keras.layers as L

def attention_gate(g, x, num_filters):
    # Linear transformation of the gating signal
    Wg = L.Conv2D(num_filters, (1, 1), padding="same")(g)
    Wg = L.BatchNormalization()(Wg)

    # Linear transformation of the skip connection signal
    Wx = L.Conv2D(num_filters, (1, 1), padding="same")(x)
    Wx = L.BatchNormalization()(Wx)
    # Adding the transformed features together and applying ReLU activation
    out = L.Activation("relu")(L.Add()([Wg, Wx]))

    # Transformation to compute the attention coefficients with sigmoid activation
    out = L.Conv2D(1, (1, 1), padding="same")(out)
    out = L.Activation("sigmoid")(out)
    # Multiplying the attention coefficients with the original signal to get the attended signal
    attended = L.Multiply()([out, x])

    return attended
def upsample_block(filters, squeeze, expand, strides, deconv_ksize, att_filters, inputs, skip_connection):
  upconv = tf.keras.layers.Conv2DTranspose(filters, deconv_ksize, strides=strides, padding='same', kernel_initializer='he_normal')
  d = upconv(inputs)

  # Attention block
  x = attention_gate(d, skip_connection, att_filters)

  # Concatenate
  d = tf.concat([x, d], axis=-1)

  # Fire module
  x = fire_module(d, squeeze, expand, expand)
  
  return x

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.layers as L

# Define the complete ATT Squeeze U-Net model
def build_att_squeeze_unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    C1 = Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same')(inputs)
    P1 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(C1)
    F2 = fire_module(P1, s1=16, e1=64, e3=64)
    F3 = fire_module(F2, s1=16, e1=64, e3=64)
    F4 = fire_module(F3, s1=32, e1=128, e3=128)
    P2 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(F4)
    F5 = fire_module(F4, s1=32, e1=128, e3=128)  
    F6 = fire_module(F5, s1=48, e1=192, e3=192)  
    F7 = fire_module(F6, s1=48, e1=192, e3=192)  
    F8 = fire_module(F7, s1=64, e1=256, e3=256)
    P3 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(F8)
    #Bridge
    F9 = fire_module(P3, s1=64, e1=256, e3=256) 
    C2 = Conv2D(2, (1, 1), activation='relu', padding='same')(F9) 
    C3 = Conv2D(1, (3, 3), activation='relu', padding='same')(C2)
    # Decoder
    # Attention Gates & DeFire Modules
#     DeFire1 = desire_module(C3, s1=64, e3=256)
#     CO1 = Concatenate(name='concat_with_ag1')([AG1, DeFire1])
    upsampling_1 = upsample_block(filters=192, squeeze=48, expand=192, strides=(1, 1), deconv_ksize=3, att_filters=96,inputs=C3,skip_connection=P3)
    C4 = Conv2D(1, (2, 2), activation='relu', padding='same')(upsampling_1)
#     AG2 = attention_gate(g=F3, x=C4, num_filters=128)  
#     DeFire2 = defire_module(AG2, s1=32, e3=128)
#     CO2 = Concatenate(name='concat_with_ag1')([AG2, DeFire2])
    upsampling_2 = upsample_block(filters=128, squeeze=32, expand=128, strides=(1, 1), deconv_ksize=3, att_filters=64,inputs=C4,skip_connection=P2)
    C5 = Conv2D(1, (3, 3), activation='relu', padding='same')(upsampling_2)
#     AG3 = attention_gate(g=C1, x=C5, num_filters=64) 
#     DeFire3 = defire_module(AG3, s1=16, e3=64)
#     CO3 = Concatenate(name='concat_with_ag1')([AG3, DeFire3])
    upsampling_2 = upsample_block(filters=64, squeeze=16, expand=64, strides=(2, 2), deconv_ksize=3, att_filters=16,inputs=C5,skip_connection=P1)
    C6 = Conv2D(64, (3, 3), activation='relu', padding='same')(upsampling_2)

    # Output
    C7 = Conv2D(2, (1, 1), activation='relu', padding='same')(C6)

    model = Model(inputs=[inputs], outputs=[C3])
    return model


input_shape = (254, 254, 3) 
model = build_att_squeeze_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
import cv2
import numpy as np
resized_images = []
for img in images_loc:
    resized_img = cv2.resize(img, (254,254))
    resized_images.append(resized_img)
resized_images = np.array(resized_images)
X_train, X_testy, y_train, y_testy = train_test_split(resized_images, labels, test_size = 0.3)
X_val, X_test, y_val, y_test = train_test_split(X_testy, y_testy, test_size = 0.5)

In [ ]:
show_25_images(X_train)

In [ ]:
test_image = resized_images[3]
test_image.shape

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
model.fit(X_train, y_train,
             batch_size=32,epochs=25,
             validation_data=(X_val, y_val),
             callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
from PIL import Image
segmentation_mask = model.predict(test_image)

# Threshold the segmentation mask to get binary mask
threshold = 0.5  # You can adjust the threshold based on your requirements
binary_mask = (segmentation_mask > threshold).astype(np.uint8)

# Save the segmented image
binary_mask_image = Image.fromarray(binary_mask[0, :, :, 0] * 255)
binary_mask_image.save('segmented_image.jpg')

In [ ]:
import matplotlib.pyplot as plt

# Load the original test image
original_image = Image.open("/kaggle/working/segmented_image.jpg")

# Display the original test image and the segmented image side by side
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(test_image)
axes[0].set_title('Original Image')

axes[1].imshow(original_image)  # Display the segmented image
axes[1].set_title('Segmented Image')

plt.show()
